In [3]:
import chitchat_dataset as data
import pandas as pd
import re
import json
import pysolr 
from tqdm import tqdm
import requests
import json

In [2]:
CORE_NAME = 'chitchat'
solr_url = f'http://localhost:8983/solr/'

In [3]:
def add_fields():
    data = {
        "add-field": [
            {
                    "name": "input",
                    "type": "text_en",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "response",
                    "type": "text_en",
                    "indexed": True,
                    "multiValued": False
                }
            ]
        }

    print(requests.post(solr_url + CORE_NAME + "/schema", json=data).json())

In [4]:
dataset = data.Dataset()
connection = pysolr.Solr(solr_url + CORE_NAME, always_commit=True, timeout=50000)

In [6]:
chit_chat_jsons =[]
for convo_id, convo in tqdm(dataset.items()):
    c=''
    p=''
    msgs = convo['messages']
    pair={'input':'','response':''}
    for msgs in convo['messages']:
        for msg in msgs:
              c=c+msg['text']+' '
        if p!='':
            pair['input'] = p.strip()
            pair['response'] = c.strip()
            chit_chat_jsons.append(dict(pair))

        p=c
        c=''
        
connection.add(chit_chat_jsons)

100%|███████████████████████████████████████████████████████████████████████████| 7168/7168 [00:00<00:00, 46012.39it/s]


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":5589}}\n'

In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
query ='i am happy'
qembeddings = model.encode(query, convert_to_tensor=True)
result = requests.get('http://localhost:8983/solr/chitchat/select?indent=true&q.op=OR&q=input%3A'+query+'&rows=100')

In [9]:
response = None
score = None
matchedBest =None
for doc in result.json()['response']['docs']:
    inp = doc['input'][0]
    inpemb= model.encode(inp, convert_to_tensor=True)
    cosine_score = util.cos_sim(inpemb, qembeddings)
    if score == None or cosine_score > score:
        score = cosine_score
        matchedBest = inp
        response = doc['response'][0]

print('query : '+query)
print('Just Solr match : '+ result.json()['response']['docs'][0]['input'][0])
print('Embedding match : '+ matchedBest)
print('Response : '+response)

query : i am happy
Just Solr match : i think i did i messaged you
Embedding match : I am not I wish
Response : Oh


In [44]:
matchedBest

'How are you doing!? How is work?'

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [17]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [19]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding))
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: 384

Sentence: Sentences are passed as a list of string.
Embedding: 384

Sentence: The quick brown fox jumps over the lazy dog.
Embedding: 384



In [49]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Single list of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'Do you like pizza?']

q = 'The new movie is so great'
q='sauce'
#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)
qembeddings = model.encode(q, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, qembeddings)

cmax = cosine_scores[0]
res = sentences[0]
for i in range(0,len(cosine_scores)):
    score = cosine_scores[i]
    if score > cmax:
        res = sentences[i]
        cmax=score
        
print(res)

I love pasta


In [45]:
cosine_scores

tensor([[ 0.4182],
        [-0.0030],
        [ 0.3094],
        [ 0.0918],
        [ 0.3564],
        [ 0.0690],
        [ 0.1192]])